<a href="https://colab.research.google.com/github/sharifkaiser/DL_colab/blob/master/board_o_d.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow-object-detection-api

     |████████████████████████████████| 583kB 4.8MB/s 
     |████████████████████████████████| 61kB 6.4MB/s 
     |████████████████████████████████| 2.3MB 57.0MB/s 
  Created wheel for tensorflow-object-detection-api: filename=tensorflow_object_detection_api-0.1.1-cp36-none-any.whl size=844515 sha256=c4c0213bf2bb18d948bda8a20c09946b0149593260d5b666f2492e1365fab7c2
  Stored in directory: /root/.cache/pip/wheels/4a/54/d0/cfca11930c4b2025d40dede77059094070a67cc3e7bd3b285f
Successfully built tensorflow-object-detection-api


In [ ]:
"""
Usage:
  # From tensorflow/models/
  # Create train data:
  python generate_tfrecord.py --csv_input=data/train_labels.csv  --output_path=train.record
  # Create test data:
  python generate_tfrecord.py --csv_input=data/test_labels.csv  --output_path=test.record
"""
from __future__ import division
from __future__ import print_function
from __future__ import absolute_import

import os
import io
import pandas as pd
import tensorflow as tf

from PIL import Image
from object_detection.utils import dataset_util
from collections import namedtuple, OrderedDict

In [ ]:
# strategy: I have uploaded the csv file to google drive, then mounted 
# drive in colab and then converted csv file into images
# mount google drive in colab
from google.colab import drive
drive.mount('/content/drive') # this is saved under /content/drive dir

print(tf.__version__) # 1.15.0 is good for edgetpu

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
1.15.0


# New Section

In [ ]:
flags = tf.app.flags
flags.DEFINE_string('csv_input', '', 'Path to the CSV input')
flags.DEFINE_string('output_path', '', 'Path to output TFRecord')
flags.DEFINE_string('image_dir', '', 'Path to images')
FLAGS = flags.FLAGS

In [ ]:
# label map
def class_text_to_int(row_label):
    if row_label == 'AvNetMicrozed':
        return 1
    if row_label == 'AvNetMicrozed_back':
        return 2
    if row_label == 'CSOC-Camera-Mdoule-P1,Rev1':
        return 3
    if row_label == 'CSOC-Camera-Mdoule-P1,Rev1_back':
        return 4
    if row_label == 'CSOC-Camera-Mdoule-P2,Rev.1':
        return 5
    if row_label == 'CSOC-Camera-Mdoule-P2,Rev.1_back':
        return 6
    if row_label == 'E627280':
        return 7
    if row_label == 'E627280_back':
        return 8
    if row_label == 'E627797':
        return 9
    if row_label == 'E627797_back':
        return 10
    if row_label == 'Raspberry-pi-b+':
        return 11
    if row_label == 'Raspberry-pi-b+_back':
        return 12
    if row_label == 'ZYBO-Z7':
        return 13
    if row_label == 'ZYBO-Z7_back':
        return 14
    else:
        None

In [ ]:
def split(df, group):
    data = namedtuple('data', ['filename', 'object'])
    gb = df.groupby(group)
    return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]


def create_tf_example(group, path):
    with tf.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size

    filename = group.filename.encode('utf8')
    image_format = b'jpg'
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        xmins.append(row['xmin'] / width)
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        classes_text.append(row['class'].encode('utf8'))
        classes.append(class_text_to_int(row['class']))

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example


# def main(_):
#     writer = tf.python_io.TFRecordWriter(FLAGS.output_path)
#     path = os.path.join(FLAGS.image_dir)
#     examples = pd.read_csv(FLAGS.csv_input)
#     grouped = split(examples, 'filename')
#     for group in grouped:
#         tf_example = create_tf_example(group, path)
#         writer.write(tf_example.SerializeToString())

#     writer.close()
#     output_path = os.path.join(os.getcwd(), FLAGS.output_path)
#     print('Successfully created the TFRecords: {}'.format(output_path))


# if __name__ == '__main__':
#     tf.app.run()

In [ ]:
# generate training tfrecord
writer = tf.python_io.TFRecordWriter('/content/drive/My Drive/board_detection/train.record')
path = os.path.join('/content/drive/My Drive/board_detection/train')
examples = pd.read_csv('/content/drive/My Drive/board_detection/train_labels.csv')
grouped = split(examples, 'filename')
for group in grouped:
    tf_example = create_tf_example(group, path)
    writer.write(tf_example.SerializeToString())

writer.close()
output_path = os.path.join(os.getcwd(), FLAGS.output_path)
print('Successfully created the TFRecords: {}'.format(output_path))

Successfully created the TFRecords: /content/


In [ ]:
# generate test tfrecord
writer = tf.python_io.TFRecordWriter('/content/drive/My Drive/board_detection/test.record')
path = os.path.join('/content/drive/My Drive/board_detection/test')
examples = pd.read_csv('/content/drive/My Drive/board_detection/test_labels.csv')
grouped = split(examples, 'filename')
for group in grouped:
    tf_example = create_tf_example(group, path)
    writer.write(tf_example.SerializeToString())

writer.close()
output_path = os.path.join(os.getcwd(), FLAGS.output_path)
print('Successfully created the TFRecords: {}'.format(output_path))

Successfully created the TFRecords: /content/


In [ ]:
# ! export PYTHONPATH=$PYTHONPATH:pwd:pwd/slim
!python model_main.py --logtostderr --model_dir=/content/drive/My Drive/board_detection/output_model/ --pipeline_config_path=/content/drive/My Drive/board_detection/mobilenet_ssd_v2_coco_board.config
# !ls
# model_dir is output model directory

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Traceback (most recent call last):
  File "model_main.py", line 26, in <module>
    from object_detection import model_lib
  File "/usr/local/lib/python3.6/dist-packages/object_detection/model_lib.py", line 28, in <module>
    from object_detection import exporter as exporter_lib
  File "/usr/local/lib/python3.6/dist-packages/object_detection/exporter.py", line 24, in <module>
    from object_detection.builders import model_builder
  File "/usr/local/lib/python3.6/dist-packages/object_detection/builders/model_builder.py", line 35, in <module>
    from object_detection.models import faster_rcnn_inception_resnet_v2_feature_extract